In [ ]:
#import all neccessary libraries
import os
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np; np.random.seed(42)
from pathlib import Path
from io import StringIO 
import io
import os
from matplotlib_venn import venn2, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles, venn3_unweighted
from mpl_toolkits import axes_grid1
import matplotlib.ticker as ticker

In [ ]:
###Coverage of target region calculation in AS and WGS###

AS = pd.read_csv("results/AS_sec.target.regions.bed.gz"
                     , sep="\t",names=["chr","start","stop","name","cov"])
WG = pd.read_csv("results/WGS_sec.regions.bed.gz"
                     , sep="\t",names=["chr","start","stop","name","cov"])

AS.drop_duplicates(inplace=True)
WG.drop_duplicates(inplace=True)

#set chromosome size#
from collections import OrderedDict
s = """chr1	248956422
chr2	242193529
chr3	198295559
chr4	190214555
chr5	181538259
chr6	170805979
chr7	159345973
chr8	145138636
chr9	138394717
chr10	133797422
chr11	135086622
chr12	133275309
chr13	114364328
chr14	107043718
chr15	101991189
chr16	90338345
chr17	83257441
chr18	80373285
chr19	58617616
chr20	64444167
chr21	46709983
chr22	50818468
chrX	156040895
chrY	57227415""".split("\n")
chr_sizes = OrderedDict(l.split("\t") for l in s)
mpl.style.use('classic')

ASdf = AS.sort_values(by=["chr",'start'])
WGdf = WG.sort_values(by=["chr",'start'])
# Plot configuration
mpl.rcParams.update(
    {
        "font.size": 10,
        "figure.facecolor": "w",
        "axes.facecolor": "w",
        "axes.spines.right": False,
        "axes.spines.top": False,
        "xtick.top": False,
        "xtick.bottom": False,
        "ytick.right": False,
        "ytick.left": False,
        "figure.dpi": 400})
lastsize = 0
chr_offset=dict()
for chr in chr_sizes:
    print (chr,lastsize,chr_sizes[chr])
    chr_offset[chr]=lastsize
    lastsize+=int(chr_sizes[chr])

ASdf['chrom']=ASdf['chr'].str.replace('chr','')#.astype(int)

ASdf = ASdf.sort_values(by=["chrom",'start'])
ASdf['ind2']=ASdf['start'].cumsum()
ASdf['ind'] = range(len(ASdf))
ASdf["chr_offset"]=ASdf['chr'].map(chr_offset)
ASdf['offsetstart']=(ASdf['stop']-ASdf['start'])/2 + ASdf['start'] +ASdf['chr_offset']
ASdf_grouped2 = ASdf.groupby(('chrom'))

WGdf['chrom']=WGdf['chr'].str.replace('chr','')#.astype(int)

WGdf = WGdf.sort_values(by=["chrom",'start'])
WGdf['ind2']=WGdf['start'].cumsum()
WGdf['ind'] = range(len(WGdf))
WGdf["chr_offset"]=WGdf['chr'].map(chr_offset)
WGdf['offsetstart']=(WGdf['stop']-WGdf['start'])/2 + WGdf['start'] +WGdf['chr_offset']
WGdf_grouped2 = WGdf.groupby(('chrom'))

##plot the graph for AS
fig = plt.figure(figsize=(20,4))
ax = fig.add_subplot(111)
colors = ['green']#,'red','green', 'red']
x_labels = []
x_labels_pos = []
maxval= 0
for num, (name, group) in enumerate(ASdf_grouped2):
    group.plot(kind='scatter', x='offsetstart', y='cov',color=colors[num % len(colors)],alpha=1,linewidth=0,s=10,edgecolors=None,marker="o", ax=ax)
    x_labels.append(name)
    x_labels_pos.append((group['offsetstart'].iloc[0-1] - (group['offsetstart'].iloc[-1] - group['offsetstart'].iloc[0])/2))
    new_maxval=group['offsetstart'].iloc[0-1]
    if new_maxval >= maxval:
        maxval = new_maxval
ax.set_xticks(x_labels_pos)
ax.set_xticklabels(x_labels)
ax.set_xlim([0, maxval])
ax.set_ylim([0, 50])
ax.set_xlabel('Chromosome')
ax.set_ylabel('Mean Coverage')
ax.set_title('Coverage of COSMIC Panel Targets in AS')

plt.margins(0.05,0)
plt.show()


fig.savefig("resultsfig/AS_COSMIC_cov",dpi = 200)

##plot the graph for WGS
fig = plt.figure(figsize=(20,4))
ax = fig.add_subplot(111)
colors = ['green']#,'red','green', 'red']
x_labels = []
x_labels_pos = []
for num, (name, group) in enumerate(WGdf_grouped2):
    group.plot(kind='scatter', x='offsetstart', y='cov',color=colors[num % len(colors)],alpha=1,linewidth=0,s=10,edgecolors=None,marker="o", ax=ax)
    x_labels.append(name)
    x_labels_pos.append((group['offsetstart'].iloc[0-1] - (group['offsetstart'].iloc[-1] - group['offsetstart'].iloc[0])/2))
    new_maxval=group['offsetstart'].iloc[0-1]
    if new_maxval >= maxval:
        maxval = new_maxval
ax.set_xticks(x_labels_pos)
ax.set_xticklabels(x_labels)
ax.set_xlim([0, maxval])
ax.set_ylim([0, 50])
ax.set_xlabel('Chromosome')
ax.set_ylabel('Mean Coverage')
ax.set_title('Coverage of COSMIC Panel Targets in WGS')

plt.margins(0.05,0)
plt.show()

fig.savefig("results/fig/WGS_COSMIC_cov",dpi = 200)







In [ ]:
#Graph for Comparison of AS Target and Non-target region coverage

#Use rglob to extract target and non target coverage for AS and for loop to read in data to a dataframe
path = Path("results/mergedCP/sec_map/AS_sec")
nontarget_paths = path.rglob("*_nontarget.regions.bed.gz")
dfsn=[]
for file in nontarget_paths: 
    print(file)
    group = file.name.split(".")[0]
    print(group)
    tmp_dfn = pd.read_csv(file, sep="\t", header=None, names=["chr","start","stop","coverage"])
    tmp_dfn["gene"] = "Nil"
    tmp_dfn["group"] = group
    dfsn.append(tmp_dfn)
dfn = pd.concat(dfsn)

target_path = path.rglob("*_target.regions.bed.gz")
dfst=[]
for file in target_path: 
    print(file)
    group = file.name.split(".")[0]
    print(group)
    tmp_dft = pd.read_csv(file, sep="\t", header=None, names=["chr","start","stop","gene","coverage"])
    tmp_dft["group"] = group
    dfst.append(tmp_dft)
dft = pd.concat(dfst)

#Concatenate all dataframes
cat_df = pd.concat([dfn, dft])

#plot the graph
fig, ax =plt.subplots(ncols=1, nrows=1, figsize=(13, 10))
cat_df["Group"] = cat_df["group"].map(
    {"AS_subset_nontarget": 'Non-target Region (AS after subset extraction)', 
     "AS_total_nontarget": 'Non-target Region (AS before subset extraction)',
     "AS_subset_target": 'Target Region (AS after subset extraction)', 
     "AS_total_target": 'Target Region (AS before subset extraction)'})
graph = sns.barplot(
    data=cat_df[cat_df["chr"].ne("chrM")], 
    x="chr", 
    y="coverage", 
    hue="Group",
    ax=ax)
plt.legend(loc='upper right', title='Comparison of AS Target and Non-target region coverage') 
graph.set_xticklabels(graph.get_xticklabels(), rotation=90) #graph.set_xticklabels(rotation = 90)
graph.set_xlabel("Mean Coverage") 
graph.set_ylabel("Chromosome")
plt.show

#save
fig.savefig("results/fig/AS_TvsNT_cov",dpi = 200)


In [ ]:
#Graph for Comparison of WGS Target and Non-target region coverage

#Use rglob to extract target and non target coverage for WGS and for loop to read in data to a dataframe
path = Path("results/mergedCP/sec_map/WGS_sec")
nontarget_paths = path.rglob("*_nontarget.regions.bed.gz")
dfsnR=[]
for file in nontarget_paths: 
    print(file)
    group = file.name.split(".")[0]
    print(group)
    tmp_dfnR = pd.read_csv(file, sep="\t", header=None, names=["chr","start","stop","coverage"])
    tmp_dfnR["gene"] = "Nil"
    tmp_dfnR["group"] = group
    dfsnR.append(tmp_dfnR)
dfnR = pd.concat(dfsnR)

target_path = path.rglob("*_target.regions.bed.gz")
dfstR=[]
for file in target_path: 
    print(file)
    group = file.name.split(".")[0]
    print(group)
    tmp_dftR = pd.read_csv(file, sep="\t", header=None, names=["chr","start","stop","gene","coverage"])
    tmp_dftR["group"] = group
    dfstR.append(tmp_dftR)
dftR = pd.concat(dfstR)

#Concatenate all dataframes
catR_df = pd.concat([dfnR, dftR])

#Graph for Comparison of AS Target and Non-target region coverage
fig, ax =plt.subplots(ncols=1, nrows=1, figsize=(13, 10))
catR_df["Group"] = catR_df["group"].map(
    {"WGS_subset_nontarget": 'Non-target Region (WGS after subset extraction)', 
     "WGS_total_nontarget": 'Non-target Region (WGS before subset extraction)',
     "WGS_subset_target": 'Target Region (WGS after subset extraction)', 
     "WGS_total_target": 'Target Region (WGS before subset extraction)'})
graph = sns.barplot(
    data=catR_df[catR_df["chr"].ne("chrM")], 
    x="chr", 
    y="coverage", 
    hue="Group",
    ax=ax)
plt.legend(loc='upper right', title='Comparison of WGS Target and Non-target region coverage') 
graph.set_xticklabels(graph.get_xticklabels(), rotation=90) #graph.set_xticklabels(rotation = 90)
graph.set_xlabel("Mean Coverage") 
graph.set_ylabel("Chromosome")
plt.show

#save the plot
fig.savefig("results/fig/WGS_TvsNT_cov",dpi = 200)


In [ ]:
###Graphs to compare basecall HAC vs SAC###

##Use panda to read in the data
basecall = pd.read_csv(
    "~/mergedCP/basecall/basecall.csv", sep='\t',
    header=None,
    skiprows=1,
    names=["Metric", "Number", "Group"],
thousands=",")


##Barplot for Basecall Accuracy comparison: number of reads
sns.set_theme(style="whitegrid")

g = sns.catplot(
    data=A, 
    kind="bar",
    x="Metric", 
    y="Number", 
    hue="Group",
    palette="dark", alpha=.8, height=5,aspect=0.8 )

g.set_xticklabels([])
g.set(xlabel="Basecall Accuracy") 
g.set(ylabel="Number of reads")
g.set(yticks=np.arange(0,275000,25000))
g.fig.suptitle("Basecall Accuracy comparison for number of reads",fontsize=11, fontdict={"weight": "bold"})
g.fig.savefig("results/fig/basecall1",dpi = 200)



##Barplot for Basecall Accuracy comparison: number of bases
B = (basecall[basecall["Metric"] == "Total bases"]) 
sns.set_theme(style="whitegrid")

g = sns.catplot(
    data=B, 
    kind="bar",
    x="Metric", 
    y="Number", 
    hue="Group",
   palette="dark", alpha=.8, height=8,aspect=0.4 )
g.set(xlabel="Basecall Accuracy") 
g.set(ylabel="Total number of bases")
g.set_xticklabels([])
g.set(yticks=np.arange(0,2.5e9,0.05e9))
g.legend.set_title("")
g.fig.suptitle("Basecall Accuracy comparison for total number of bases",fontsize=11, fontdict={"weight": "bold"}, va = 'bottom')
#save figure
g.fig.savefig("results/fig/basecall2",dpi = 200)



##Barplot for Basecall Accuracy comparison: Quality of reads
#Read in the data with pandas
QualityYY = pd.read_csv(
    "~/mergedCP/basecall/quality", sep='\t',
    header=None,
    skiprows=1,
    names=["Quality_cutoffs",	"HAC",	"SUP",	"HAC%",	"SAC%",	"HAC_Reads","SAC_Reads"],
thousands=",")
QualityYY

#Generat plot
A = QualityYY[["Quality_cutoffs",	"HAC",	"SUP"]]
B = pd.melt(A, id_vars=['Quality_cutoffs'], value_vars=["HAC","SUP"],var_name='Accuracy')

#Plot graph
sns.set_theme(style="whitegrid")
g = sns.catplot(
    data=B, kind="bar",
    x='Quality_cutoffs',y = 'value', hue='Accuracy',
    ci="sd", palette="dark", alpha=.8, height=5
)
g.despine(left=True)
g.set_axis_labels("", "Number of reads")
g.legend.set_title("")
g.fig.suptitle('Number of reads above quality cutoffs',fontsize=11, fontdict={"weight": "bold"})
#save figure
g.fig.savefig("results/fig/basecall3",dpi = 200)



In [ ]:
###Venn diagram for HAC and SUP comparison against GIAB truth set ### 

#Define a function (read_vcf) to read in vcf
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'WGS': str, 'ALT': str,}, #'ID': str, 'WGS': str, 'ALT': str,'QUAL': str, 'FILTER': str, 'INFO': str}
        sep='\t',usecols = ["#CHROM", "POS", "WGS", "ALT"]
    ).rename(columns={'#CHROM': 'CHROM'})
# @YoavEtzioni

##Read in sup vcfs and intersections with read_vcf function
from pathlib import Path
path= "results/mergedCP/pie_sac"
search_path = Path(path)
dfs=[]
for file in search_path.glob("*.vcf"):
    group = file.name.split(".")[0]
    vcf = read_vcf(file)
    vcf["group"] = group
    dfs.append(vcf)
df = pd.concat(dfs)
df.reset_index(inplace=True,drop=True)  

#Plot venn diagrams for Super accuracy vs GIAB / GIAB vs AS - HIGH CONFIDENCE REGIO
C = df.group.value_counts().Both
A = df.group.value_counts().clair3_HC
A1= A - C
B = df.group.value_counts().GIAB_HC
B1 = B - C
v = venn2_unweighted(subsets = (A1, B1, C ),
      set_labels = ('AS','GIAB'))                   
fig = plt.gcf()
fig.set_facecolor("w")
plt.show
plt.title ('SNP called from guppy Super accuracy basecaller output')
fig.savefig("results/fig/basecall4",dpi = 200)

###Read in hac vcfs and intersections with read_vcf function
from pathlib import Path
path= "results/mergedCP/pie2_hac/"
search_path = Path(path)
dfs=[]
for file in search_path.glob("*.vcf"):
    group = file.name.split(".")[0]
    vcf = read_vcf(file)
    vcf["group"] = group
    dfs.append(vcf)
dfh = pd.concat(dfs)
dfh.reset_index(inplace=True,drop=True)  

#plot venn diagrams for High accuracy vs GIAB
C = dfh.group.value_counts().Both
A = dfh.group.value_counts().clair3_HC
A1= A - C
B = dfh.group.value_counts().GIAB_HC
B1 = B - C

v=venn2_unweighted(subsets = (A1, B1, C ),
      set_labels = ('AS','GIAB'))
fig = plt.gcf()
fig.set_facecolor("w")    
plt.title ('SNP called from guppy High accuracy basecaller output')
plt.show
fig.savefig("results/fig/basecall5",dpi = 200)


#plot venn diagrams GIAB vs WGS - HIGH CONFIDENCE REGION'
C = dfall.group.value_counts().GIAB_isect_WGS
A = dfall.group.value_counts().GIAB
A1= A - C
B = dfall.group.value_counts().NA12878
B1 = B - C

v=venn2_unweighted(subsets = (A1, B1, C ),
      set_labels = ('GIAB', 'WGS'))

fig = plt.gcf()
fig.set_facecolor("w")    
plt.title ('GIAB vs WGS - HIGH CONFIDENCE REGION')
plt.show

fig.savefig("results/fig/AS_WGS_HC",dpi = 200)


#plot venn diagrams GIAB vs WGS vs AS - HIGH CONFIDENCE REGION'
A = dfall.group.value_counts().GIAB
B = dfall.group.value_counts().AS
C = dfall.group.value_counts().GIAB_isect_AS
D = dfall.group.value_counts().WGS
E = dfall.group.value_counts().GIAB_isect_WGS
F = dfall.group.value_counts().WGS_isect_AS
G = dfall.group.value_counts().All_three_isect
C1 = C - G
F1 = F - G
E1 = E - G
A1 = A - (C1 + E1 + G)
B1 = B - (C1 + F1 + G)
D1 = D - (E1 + F1 + G)

v=venn3_unweighted(subsets=(A1, B1, C1, D1, E1 ,F1 ,G), set_labels = ('GIAB', 'AS', 'WGS'))

fig = plt.gcf()
fig.set_facecolor("w")
plt.title ('Comparison of SNP called at High Confidence region in GIAB, AS, WGS')
plt.show()
fig.savefig("results/fig/AS_GIAB_WGS_HC",dpi = 200)



In [ ]:
###Graph to plot Distribution of AS coverage in AS only, GIAB only and both###
##Read in the table with pandas
giab = pd.read_csv(
    "results/mergedCP/variant_calling/gatktables/GIAB.table", sep='\t',
    header=None,
    usecols=[0, 1, 3],
    names=["chrom", "pos", "depth"])
giab["group"] = "GIAB"
Clair3 = pd.read_csv(
    "results/mergedCP/variant_calling/gatktables/AS.table", sep='\t',
    skiprows=1,
    header=None,
    names=["chrom", "pos", "depth"])
Clair3["group"] = "AS"
Both = pd.read_csv(
    "results/mergedCP/variant_calling/gatktables/Both.table", sep='\t',
    skiprows=1,
    header=None,
    names=["chrom", "pos", "depth"])
Both["group"] = "BOTH"
#concatenate the table
dfHC_2 = pd.concat([giab, Clair3, Both])

#Plot the graph
fig, ax =plt.subplots(ncols=1, nrows=1, figsize=(8, 6))

ax = sns.violinplot(
    data = dfHC_2,
    x="group", 
    y="depth")
ax.set_title("Distribution of coverage")
ax.set_ylabel("Depth")
ax.set_xlabel("")
plt.show()

fig.savefig("results/fig/AS_GIAB_COV",dpi = 200)

In [ ]:
WGS_meth = pd.read_csv(
    "results/whole_genome/5mC_NA12878_bedmethyl", sep='\t',
    header=None,
    names=["chrom", "start", "end", "name", "score", "strand", "tstart", "tend", "color", "coverage", "freq", "ref_G", "sample_G", "qual"])
AS_meth = pd.read_csv(
    "results/methylation_mcp/modbam2bed/5mC_ml32merged_bedmethyl", sep='\t',
    header=None,
    names=["chrom", "start", "end", "name", "score", "strand", "tstart", "tend", "color", "coverage", "freq", "ref_G", "sample_G", "qual"])
meth_merged = pd.merge(WGS_meth, AS_meth, how="inner", on=["chrom","start","end"], suffixes = ("_WGS","_AS"))
methyl


POLD1 = methyl[methyl['chrom'].eq('chr19') & methyl['start'].between(50384340,50418010)]
FREQ_POLD1 = POLD1[["WGS_meth","AS_meth"]]
matrix = FREQ_POLD1.corr()
print(matrix)